Adaptacion de modelo tomado de https://apmonitor.com/dde/index.php/Main/LLMTransformers con datos de Shanghai

In [1]:
#Carguemos los datos
import pandas as pd
ShanghaiDat=pd.read_csv('Shanghai-2005-2025.csv')
ShanghaiDat=ShanghaiDat.drop(ShanghaiDat.columns[0], axis=1) #Eliminar la primera columna que tiene la etiqueta

print('Dimension del dataframe:',ShanghaiDat.shape)
print('Primer elemento del dataframe:',ShanghaiDat.iloc[0,0])

Dimension del dataframe: (1, 4875)
Primer elemento del dataframe: 1242.7740478515625


In [2]:
#Nos saltamos la parte del codigo que genera datos porque nosotros ya tenemos datos reales
#Debemos convertir nuestros datos en una ventana de datos deslizantes. Esto es, obtener una matriz dividiendo los datos en epocas de 10 dias y obtener los target para cada una de esas epocas
import numpy as np

# Convertimos los datos a un array de 1 dimension
ShanghaiDat = np.array(ShanghaiDat).flatten()

#Dividimos en epocas de 10 dias
def Epocas(sequence_length=10):
    size = len(ShanghaiDat) 
    sequences = [ShanghaiDat[i:i+sequence_length] for i in range(size-sequence_length)]
    next_points = ShanghaiDat[sequence_length:]

    # Convertir a NumPy arrays correctamente
    return np.array(sequences), np.array(next_points)

Shanghai_epocas, Shanghai_targets = Epocas()

print('Shape de Shanghai_epocas:', Shanghai_epocas.shape)
print('Shape de Shanghai_targets:', Shanghai_targets.shape)

#Ahora tenemos 4865 epocas de 10 dias en una variable, y en la otra varibale todos los targets de cada epoca. Es importante señalar que ninguna epoca puede contener a su target
#Es por ello que en la ultima epoca solo se llega al penultimo valor
print(Shanghai_epocas[-1])

Shape de Shanghai_epocas: (4865, 10)
Shape de Shanghai_targets: (4865,)
[3236.03198242 3241.82104492 3244.37792969 3242.62304688 3213.62402344
 3230.1640625  3252.62597656 3250.60107422 3229.48803711 3270.65893555]


In [3]:
#Despues necesitamos definir una clase que permita tratar los datos para poder usar DataLoader de pytorch.
#Terminamos de cargar todas las librerias que vamos a usar
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

class TimeSeriesDataset(Dataset): #Heredamos caracteristicas de Dataset
    def __init__(self, sequences, next_points): #Determina que parametros debemos pasar la crear una instancia
        self.sequences = sequences
        self.next_points = next_points

    def __len__(self): #Nos dice la longitud de nuestros datos de secuancia
        return len(self.sequences)

    def __getitem__(self, idx): #Nos da cualquier epoca que queramos y su targeta asociado
        return self.sequences[idx], self.next_points[idx]

In [4]:
#Probamos crear una instancia con nuestros datos
Datos=TimeSeriesDataset(Shanghai_epocas, Shanghai_targets)
print(len(Datos))
Datos[-1]

4865


(array([3236.03198242, 3241.82104492, 3244.37792969, 3242.62304688,
        3213.62402344, 3230.1640625 , 3252.62597656, 3250.60107422,
        3229.48803711, 3270.65893555]),
 np.float64(3303.6669921875))

In [5]:
# Transformer Model (simplified for numerical data)
class TransformerModel(nn.Module):
    def __init__(self, input_size=1, sequence_length=10, num_layers=1, num_heads=2, dim_feedforward=512): #Se definen los parametros del modelo. Una capa, 2 cabezas de atencion
        super(TransformerModel, self).__init__() #Llama al init de la clase padre, que es el constructor del modelo. Es necesario para crear modelos personalizados
        self.sequence_length = sequence_length
        self.encoder_layer = nn.TransformerEncoderLayer(d_model=input_size*sequence_length, nhead=num_heads, dim_feedforward=dim_feedforward) #Define una capa
        self.transformer_encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=num_layers) #Crea el transformer con definido anteriormente
        self.fc_out = nn.Linear(input_size * sequence_length, 1) #Capa fully connected que toma la salida del trandormer (15 valores) y regresa un unico valor (prediccion, next point)

    
    #Aqui se define como fluye la informacion a traves del modelo. Pasar datos por la red y obtener predicciones
    def forward(self, src): #src es el tensor de entrada
        # Reshape to match the input dimensions
        src = src.reshape(-1, self.sequence_length, 1)  #-1 hace que se ajuste automaticamente el tamaño del batch
        src = src.flatten(start_dim=1) #Aplana la dimension 1D
        src = src.unsqueeze(0)  # Add batch dimension *
        out = self.transformer_encoder(src) #Aplica el modelo en la secuencia
        out = out.squeeze(0)  # Remove batch dimension *
        return self.fc_out(out) #Le aplica la capa de salida

In [6]:
# Preparamos los datos
sequences, next_points = Epocas() #Cargamos nuestras epocas de 15 dias(Shanghai_epocas) y nuestros target (Shanghai_targets)
dataset = TimeSeriesDataset(sequences, next_points) #Asignamos las funciones a nuestros datos para poder manipularlos
dataloader = DataLoader(dataset, batch_size=32, shuffle=True) #Divide los datos en batches y los revuelve

In [7]:
#Entrenamiento del modelo
model = TransformerModel() #Se llama al modelo definido
criterion = nn.MSELoss() #Funcion de perdida
optimizer = torch.optim.Adam(model.parameters(), lr=0.001) #Optimizador

# Training loop
for epoch in range(20):  # Numero de epocas, esto es numero de veces que todos los datos pasan por el modelo
    for seq, next_point in dataloader: #Itera a traves de los batches
        seq, next_point = seq.float(), next_point.float().unsqueeze(1) #Pone los datos en forma decimal 
        output = model(seq) #Se pasan las secuencias al modelo para obtener la prediccion output
        loss = criterion(output, next_point) #Se calcula la perdida 
        optimizer.zero_grad() #Se limpian los gredientes de la iteracion anterior
        loss.backward() #Back propagation. Gradientes de la perdida con respecto a los pesos y bias
        optimizer.step() #Actualiza los parametros del modelo con base en los gradientes
    print(f"Epoch {epoch+1}, Loss: {loss.item()}")

C:\Users\Antonio Rojas\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torch\nn\modules\transformer.py:385: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


Epoch 1, Loss: 1670206.0
Epoch 2, Loss: 12443139.0
Epoch 3, Loss: 5482061.0
Epoch 4, Loss: 1488125.75
Epoch 5, Loss: 5165007.5
Epoch 6, Loss: 10512810.0
Epoch 7, Loss: 3198289.5
Epoch 8, Loss: 9082861.0
Epoch 9, Loss: 4491235.0
Epoch 10, Loss: 9394131.0
Epoch 11, Loss: 4991873.0
Epoch 12, Loss: 3776616.75
Epoch 13, Loss: 7387838.5
Epoch 14, Loss: 8273649.5
Epoch 15, Loss: 7123804.5
Epoch 16, Loss: 7048106.0
Epoch 17, Loss: 2852552.75
Epoch 18, Loss: 2996812.0
Epoch 19, Loss: 8560142.0
Epoch 20, Loss: 8639546.0


In [8]:
#Veamos como predice el modelo
test_seq = torch.tensor(sequences[0]).float()
predicted_point = model(test_seq)
print('Valores usados para la prediccion:',sequences[0])
print("Prediccion(dato siguiente):", predicted_point.item())
print('Dato siguiente real:',sequences[1,0])

Valores usados para la prediccion: [1242.77404785 1251.93701172 1239.43005371 1244.74597168 1252.40100098
 1257.46203613 1256.92297363 1256.31396484 1245.61999512 1216.65197754]
Prediccion(dato siguiente): 394.0807189941406
Dato siguiente real: 1251.93701171875
